In [ ]:
using  Gmsh: gmsh
using  GridapGmsh
using  Gridap
using  Gridap.Geometry
using  Gridap.TensorValues
using  PyPlot

In [ ]:
const E = 5.9827e3
const ν = 0.22
const Gc = 2.28
const ls = 0.45
const η = 1e-15

In [ ]:
const L = 65      #Length
const H = 120      #Height
const Lh = 10     #notch length
const CP = 55      #center height  
const CH = H/40    #Crack height
const Lu = 28.5      #offset of hole from right end
const Ld = 51      #offset of hole from bottom end
const hfc = ls/50
const hf = ls/5   #finer mesh size
const h = 100*hf     #coarser mesh size
const di = 20      # Diameter of hole
const FMR = 7*ls  

gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 1)
p1 = gmsh.model.geo.addPoint(0.0, 0.0, 0.0, h)   
p2 = gmsh.model.geo.addPoint(L, 0, 0.0, h) 
p3 = gmsh.model.geo.addPoint(L, H, 0.0, h)
p4 = gmsh.model.geo.addPoint(0, H, 0.0, h)
p5 = gmsh.model.geo.addPoint(0, H-CP, 0.0, h)
p6 = gmsh.model.geo.addPoint(Lh, H-CP, 0.0, hfc)
p7 = gmsh.model.geo.addPoint(Lh, H-CP-0.5*CH, 0.0, hfc)
p8 = gmsh.model.geo.addPoint(0, H-CP-0.5*CH, 0.0, h)

p9 = gmsh.model.geo.addPoint(L-Lu-0.5*di, Ld, 0.0, h)
p10 = gmsh.model.geo.addPoint(L-Lu, Ld, 0.0, h)
p11 = gmsh.model.geo.addPoint(L-Lu+0.5*di, Ld, 0.0, h)

p12 = gmsh.model.geo.addPoint(L-Lu, Ld+0.5*di, 0.0, h)

l1 = gmsh.model.geo.addLine(p1, p2)
l2 = gmsh.model.geo.addLine(p2, p3)
l3 = gmsh.model.geo.addLine(p3, p4)
l4 = gmsh.model.geo.addLine(p4, p5)
l5 = gmsh.model.geo.addLine(p5, p6)
l6 = gmsh.model.geo.addLine(p6, p7)
l7 = gmsh.model.geo.addLine(p7, p8)
l8 = gmsh.model.geo.addLine(p8, p1)

l9 = gmsh.model.geo.addLine(p7, p9)
l10 = gmsh.model.geo.addLine(p7, p12)

c1 = gmsh.model.geo.addCircleArc(p9,p10,p11)
c2 = gmsh.model.geo.addCircleArc(p11,p10,p9)

cl1 = gmsh.model.geo.addCurveLoop([l1,l2,l3,l4,l5,l6,l7,l8])
cl2 = gmsh.model.geo.addCurveLoop([c1,c2])

ps1 = gmsh.model.geo.addPlaneSurface([cl1,-cl2])

pg1 = gmsh.model.addPhysicalGroup(2, [ps1])
pg2 = gmsh.model.addPhysicalGroup(1, [l1])
pg3 = gmsh.model.addPhysicalGroup(1, [l3])

gmsh.model.setPhysicalName(2, pg1, "Domain")
gmsh.model.setPhysicalName(1, pg2, "DirichletBot")
gmsh.model.setPhysicalName(1, pg3, "DirichletTop")


gmsh.model.mesh.field.add("Box", 11)
gmsh.model.mesh.field.setNumber(11, "VIn", hf)
gmsh.model.mesh.field.setNumber(11, "VOut", h)
gmsh.model.mesh.field.setNumber(11, "XMin", L-Lu+0.45*di)
gmsh.model.mesh.field.setNumber(11, "XMax", L)
gmsh.model.mesh.field.setNumber(11, "YMin", Ld-FMR)
gmsh.model.mesh.field.setNumber(11, "YMax", Ld+FMR)

gmsh.model.mesh.field.add("Distance", 1)
gmsh.model.mesh.field.setNumbers(1, "EdgesList", [l10])

gmsh.model.mesh.field.add("Threshold", 2)
gmsh.model.mesh.field.setNumber(2, "IField", 1)
gmsh.model.mesh.field.setNumber(2, "LcMin", hf)
gmsh.model.mesh.field.setNumber(2, "LcMax", h)
gmsh.model.mesh.field.setNumber(2, "DistMin", FMR)
gmsh.model.mesh.field.setNumber(2, "DistMax", 0.50*H)

gmsh.model.mesh.field.add("Min", 7)
gmsh.model.mesh.field.setNumbers(7, "FieldsList", [11,2])

gmsh.model.mesh.field.setAsBackgroundMesh(7)

gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(2)
gmsh.write("NotchedPlateWithHole.msh")
gmsh.finalize()

In [ ]:
model = GmshDiscreteModel("NotchedPlateWithHole.msh")
writevtk(model ,"NotchedPlateWithHole")

In [ ]:
function  ElasFourthOrderConstTensor(E,ν,PlanarState)
    # 1 for  Plane  Stress  and 2 Plane  Strain  Condition
    if  PlanarState  == 1
        C1111 =E/(1-ν*ν)
        C1122 = (ν*E)/(1-ν*ν)
        C1112 = 0.0
        C2222 =E/(1-ν*ν)
        C2212 = 0.0
        C1212 =E/(2*(1+ν))
    elseif  PlanarState  == 2
        C1111 = (E*(1-ν*ν))/((1+ν)*(1-ν-2*ν*ν))
        C1122 = (ν*E)/(1-ν-2*ν*ν)
        C1112 = 0.0
        C2222 = (E*(1-ν))/(1-ν-2*ν*ν)
        C2212 = 0.0
        C1212 =E/(2*(1+ν))
    end
    C_ten = SymFourthOrderTensorValue(C1111 ,C1112 ,C1122 ,C1112 ,C1212 ,C2212 ,C1122 ,C2212 ,C2222)
    return   C_ten
end

In [ ]:
const  C_mat = ElasFourthOrderConstTensor(E,ν,2)

In [ ]:
I2 = SymTensorValue{2,Float64}(1.0 ,0.0 ,1.0)
I4 = I2⊗I2
I4_sym = one(SymFourthOrderTensorValue{2,Float64})
P_vol = (1.0/3)*I4
P_dev = I4_sym  - P_vol

In [ ]:
function σfun(ε,ε_in,s_in)
    σ_elas = C_mat⊙ε
    if tr(ε_in) >= 0
        σ = (s_in ^2+η)*σ_elas
    elseif  tr(ε_in) < 0
        σ = (s_in ^2+η)*P_dev ⊙ σ_elas + P_vol ⊙ σ_elas
    end
    return  σ
end

In [ ]:
function ψPos(ε_in)
   σ_elas = C_mat⊙ε_in
    if tr(ε_in) >= 0
        ψPlus = 0.5*(ε_in ⊙ σ_elas)
    elseif  tr(ε_in) < 0
        ψPlus = 0.5*((P_dev ⊙ σ_elas)⊙(P_dev⊙ε_in))
    end
    return ψPlus
end

In [ ]:
function  new_EnergyState(ψPlusPrev_in,ψhPos_in)
    ψPlus_in = ψhPos_in
    if ψPlus_in  >=ψPlusPrev_in
        ψPlus_out =ψPlus_in
    else
        ψPlus_out=ψPlusPrev_in
    end
    true,ψPlus_out
end

In [ ]:
function  project(q,model ,dΩ,order)
    reffe = ReferenceFE(lagrangian ,Float64 ,order)
    V = FESpace(model ,reffe ,conformity =:L2)
    a(u,v) =∫(u*v)*dΩ
    b(v) =∫(v*q)*dΩ
    op = AffineFEOperator(a,b,V,V)
    qh = solve(op)
    return  qh
end

In [ ]:
order = 1
reffe_PF = ReferenceFE(lagrangian ,Float64,order)
V0_PF = TestFESpace(model ,reffe_PF;conformity =:H1)
U_PF = TrialFESpace(V0_PF)
sh = zero(V0_PF)

In [ ]:
reffe_Disp = ReferenceFE(lagrangian ,VectorValue{2,Float64},order)
V0_Disp = TestFESpace(model,reffe_Disp;conformity =:H1,
    dirichlet_tags = ["DirichletBot","DirichletTop"],
    dirichlet_masks =[(true ,true), (false ,true)])

uh = zero(V0_Disp)

In [ ]:
degree = 2*order
Ω= Triangulation(model)
dΩ= Measure(Ω,degree)

In [ ]:
labels = get_face_labeling(model)
LoadTagId = get_tag_from_name(labels ,"DirichletTop")
Γ_Load = BoundaryTriangulation(model ,tags = LoadTagId)
dΓ_Load = Measure(Γ_Load ,degree)
n_Γ_Load = get_normal_vector(Γ_Load)

In [ ]:
function   stepPhaseField(uh_in ,ΨPlusPrev_in)
    a_PF(s,φ) =∫( Gc*ls*∇(φ)⋅∇(s)+ 2*ΨPlusPrev_in*s*φ+ (Gc/ls)*s*φ)*dΩ
    b_PF(φ) =∫( (Gc/ls)*φ)*dΩ
    op_PF = AffineFEOperator(a_PF,b_PF ,U_PF ,V0_PF)
    sh_out = solve(op_PF)
    return  sh_out
end

In [ ]:
function  stepDisp(uh_in,sh_in ,vApp)
    uApp1(x) = VectorValue(0.0,0.0)
    uApp2(x) = VectorValue(0.0,vApp)
    U_Disp = TrialFESpace(V0_Disp ,[uApp1 ,uApp2])
    a_Disp(u,v) =∫( (ε(v)⊙(σfun∘(ε(u),ε(uh_in),sh_in)) ) )*dΩ
    b_Disp(v) = 0.0
    op_Disp = AffineFEOperator(a_Disp ,b_Disp ,U_Disp ,V0_Disp)
    uh_out = solve(op_Disp)
    return  uh_out
end

In [ ]:
vApp = 0.0
delv = 1e-2
const  vAppMax = 0.56
innerMax = 10
count = 0
Load = Float64[]
Displacement = Float64[]

push!(Load, 0.0)
push!(Displacement, 0.0)

sPrev = CellState(1.0,dΩ)
sh = project(sPrev ,model ,dΩ,order)
ψPlusPrev = CellState(0.0,dΩ)

while  vApp .< vAppMax
    count = count  .+ 1
    if vApp  >= 0.38
        delv = 5e-4
    end
    
    vApp = vApp .+ delv
    print("\n Entering  displacemtent  step :", float(vApp))
    for  inner = 1: innerMax
        ψhPlusPrev = project(ψPlusPrev ,model ,dΩ,order)
        RelErr = abs(sum(∫( Gc*ls*∇(sh)⋅∇(sh) + 2*ψhPlusPrev*sh*sh + (Gc/ls)*sh*sh)*dΩ-∫( (Gc/ls)*sh)*dΩ))/abs(sum(∫( (Gc/ls)*sh)*dΩ))
        print("\n Relative error = ",float(RelErr))
        sh = stepPhaseField(uh,ψhPlusPrev)
        uh = stepDisp(uh,sh,vApp)
        ψhPos_in =ψPos∘(ε(uh))
        update_state!( new_EnergyState ,ψPlusPrev ,ψhPos_in)
        if   RelErr  < 1e-8
            break
        end
    end
    Node_Force = sum(∫(n_Γ_Load⋅(σfun∘(ε(uh),ε(uh),sh)))*dΓ_Load)
    push!(Load , Node_Force[2])
    push!( Displacement , vApp)
    writevtk(Ω,"results_NotchedPlatewithHoleTest$count",cellfields= ["uh"=>uh ,"s"=>sh])
 end

In [ ]:
plot(Displacement,Load*1e-3)